In [1]:
# Importo librerías

import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
# Obtengo credenciales

client_credentials_manager = SpotifyClientCredentials("bcd05cdd915c49429590886e25342140", 
                                                      "0eae1090f5c347769d17b38d4892e303")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [37]:
# De la aplicación voy a recibir una lista de nombres de usuarios
# Tomo la siguiente lista como ejemplo:

users = ["sedepa", "zltm3cqxt0u0s9k4e8oy564dk"]

In [91]:
# Para el paso 3 del layout, voy a tener que devolver a cada usuario una lista con el nombre
# de sus playlists, para que seleccione cuáles va a querer subir para crear la lista conjunta.

# Por tanto, de la siguiente función voy a obtener un diccionario para devolver a la app, con
# el siguiente formato:

diccionario1 = {"user1": 
                   ["playlist1_name", "playlist2_name"],
                "user2": 
                   ["playlist1_name", "playlist2_name"]}


# Además, aunque al usuario en la app no se lo voy a mostrar, también voy a aprovechar para 
# obtener aquí el playlist_ID correspondiente a cada playlist, guardándolos en otro diccionario 
# con el siguiente formato:


diccionario2 = {"user1": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"},
                "user2": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"}}

In [92]:
# Defino la función que obtenga ambos diccionarios a partir de una lista de usuarios.

def get_user_playlists(users):
    users_playlists_names = {}
    users_playlists_names_IDs = {}
    for user in users:
        users_playlists_names_IDs[user] = {}
        playlists = sp.user_playlists(user)
        playlists_names = []
        for i, playlist in enumerate(playlists["items"]):
            playlists_names.append(playlist["name"])
            users_playlists_names_IDs[user][playlist["name"]] = playlist['uri'][17:]
        users_playlists_names[user] = playlists_names
    return users_playlists_names, users_playlists_names_IDs


In [93]:
# Guardo cada diccionario en una variable

users_playlists_names = get_user_playlists(users)[0]
users_playlists_names_IDs = get_user_playlists(users)[1]

In [94]:
# El diccionario users_playlists_names va a ser devuelto a la app para que cada usuario
# seleccione de entre sus playlists cuáles quiere subir.

# Además, cada usuario va a poder filtrar las canciones de sus playlists seleccionadas 
# por los géneros que escoja.

# También van a tener que indicar cuál de ellos será el "host" (el usuario en cuyo perfil 
# se creará la lista conjunta), y el nombre de la lista conjunta. 

In [95]:
# Después de que los usuarios hayan seleccionado las playlists que quieren subir, el género
# de las canciones, el host y el nombre de la nueva lista, tengo que recibir 
# un diccionario con el siguiente formato:

mygroup = {"users": 
                   {"user1": {"playlists": [], "genres": []}, 
                    "user2": {"playlists": [], "genres": []}}, 
           "host": "user1",
           "new_playlist_name": "name"}

# Siendo consciente de que es posible que las listas de valores de "genres" estén vacías,
# lo cual significa que el usuario no quiere filtrar por género, y quiere seleccionar 
# todas las canciones de las playlists que sube. 

In [101]:
# Tomo el siguiente grupo como ejemplo:

mygroup = {"users": 
                   {"sedepa": {"playlists": ["MAD", "Correr", "A little of verguenza, please"],
                               "genres": []}, 
                    "zltm3cqxt0u0s9k4e8oy564dk": {"playlists": ["Final Project"],
                                                  "genres": []}}, 
           "host": "zltm3cqxt0u0s9k4e8oy564dk",
           "new_playlist_name": "ÉXITO"}


In [100]:
for user in mygroup["users"].keys():
    for playlist in mygroup["users"][user]["playlists"]:
        print(user, users_playlists_names_IDs[user][playlist])

sedepa 5j5zTo5sz2ZQxalDea61pe
sedepa 1L5mlbiFpxXGm2WIbQJWpP
sedepa 0eLjCQyrbElR5CRFrfnFut
zltm3cqxt0u0s9k4e8oy564dk 6sHkEyFhxTQIDxCQOZYAns


In [102]:
# Defino una función que obtenga una lista con todas las canciones que hay en todas las 
# playlist que los usuarios me han pasado, filtrando por género en caso de que así lo
# hayan especificado.
# Para ello, primero tengo que obtener el ID correspondiente a cada nombre de playlist
# que me hayan pasado.

def get_tracks(group):
    tracks = []
    for user in mygroup["users"].keys():
        for playlist in mygroup["users"][user]["playlists"]:
            tracks.append(sp.user_playlist_tracks(user, 
                                 playlist_id = users_playlists_names_IDs[user][playlist], 
                                 fields=None, limit=100, offset=0, market=None))
    return tracks

In [103]:
print(get_tracks(mygroup))

[{'href': 'https://api.spotify.com/v1/playlists/5j5zTo5sz2ZQxalDea61pe/tracks?offset=0&limit=100', 'items': [{'added_at': '2019-05-26T21:47:20Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/sedepa'}, 'href': 'https://api.spotify.com/v1/users/sedepa', 'id': 'sedepa', 'type': 'user', 'uri': 'spotify:user:sedepa'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5Hsv8dUHHOdnn72q4XIVz7'}, 'href': 'https://api.spotify.com/v1/artists/5Hsv8dUHHOdnn72q4XIVz7', 'id': '5Hsv8dUHHOdnn72q4XIVz7', 'name': 'Leiva', 'type': 'artist', 'uri': 'spotify:artist:5Hsv8dUHHOdnn72q4XIVz7'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI',

In [104]:
# Estoy intentando encontrar la forma de filtrar por géneros:

sp.artist("3MyFDtqB80WZvbtCZRsekM?si=w_gUMqpmSnir6lEmV5UDDQ")

# He descubierto que es a través del artista.

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3MyFDtqB80WZvbtCZRsekM'},
 'followers': {'href': None, 'total': 215275},
 'genres': ['deep euro house',
  'deep house',
  'deep tropical house',
  'minimal tech house',
  'tropical house'],
 'href': 'https://api.spotify.com/v1/artists/3MyFDtqB80WZvbtCZRsekM',
 'id': '3MyFDtqB80WZvbtCZRsekM',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/4c46805193470bdfc9e448df6c13be526a59c7a7',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/23216cb30809863cf0b77fc1d5b98b9c95bd9479',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/26676a525bac1785376ac962183977304cb6f7ee',
   'width': 160}],
 'name': 'Bakermat',
 'popularity': 65,
 'type': 'artist',
 'uri': 'spotify:artist:3MyFDtqB80WZvbtCZRsekM'}

In [ ]:
def get_user_playlists(users):
    playlists = sp.user_playlists(user)
    for i, playlist in enumerate(playlists['items']):
        users_playlists.append(playlist['uri'][17:])

In [4]:
# Defino función para obtener las playlists de un usuario. En lugar de que esta función
# apendee el return a una lista de solo playlist_ids, necesito obtener también el user, para 
# después introducirlo como argumento en la función que obtiene los temas de las playlists. 
# Además de los users a juntar, esta primera función va a tener que recibir los siguientes 
# argumentos: host (usuario en cuyo perfil se creará la lista conjunta), playlist (posibilidad
# de que un usuario suba solo una/algunas playlists en lugar de todas las de su perfil), y 
# genre/s (posibilidad de que el usuario filtre las canciones a subir según su género). 
# Los argumentos "playlist" y "genre" tendrán que tener un valor predeterminado que obtenga
# todas las playlists/canciones en caso de que el usuario no especifique ninguna playlist 
# ni ningún género concreto.

def get_user_playlists(user):
    playlists = sp.user_playlists(user)
    for i, playlist in enumerate(playlists['items']):
        users_playlists.append(playlist['uri'][17:])

In [60]:
# Defino la función para obtener los temas de una playlist.

tracks = sp.user_playlist_tracks("zltm3cqxt0u0s9k4e8oy564dk", 
                                 playlist_id= "6sHkEyFhxTQIDxCQOZYAns", 
                                 fields=None, limit=500, offset=0, market=None)

In [116]:
tracks_IDs = []
tracks_names = []
tracks_artist = []

for item in tracks["items"]:
    tracks_IDs.append(item['track']["id"])
    tracks_names.append(item['track']["name"])
    for artist in item['track']["album"]["artists"]:
        tracks_artist.append(artist["name"])

In [117]:
print(len(tracks_IDs))
print(len(tracks_names))
print(len(tracks_artist))

13
13
13
